# Abstract for the Project

## Task

The task is to test which model architecture performs best at imdb sentiment classification.

## Model

The models utilize BERT embeddings and various combinations of lstm , cnn , and graph techniques.

## Experiment

The experiments are in progress. We are looking for accuracy, failure points, and interpretability. The goal is to test for full data, limited data, and limited label scenarios.

## Datasets

The main dataset is the IMDB movie reviews and whatever was used to produce the BERT embeddings.

# Code

## Import statements

In [1]:
%pip install -U --quiet tensorflow-text==2.8.1

     |████████████████████████████████| 4.9 MB 5.7 MB/s 


In [2]:
%pip install --quiet neural_structured_learning

     |████████████████████████████████| 120 kB 5.0 MB/s 


In [3]:
%pip install --quiet spektral

     |████████████████████████████████| 129 kB 4.9 MB/s 


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import neural_structured_learning as nsl
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import pandas as pd

from spektral.data import Graph
from spektral.data import Dataset
from spektral.transforms import GCNFilter
import pandas as pd
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.data.loaders import SingleLoader
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print(
    "GPU is",
    "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.2
Eager mode:  True
Hub version:  0.12.0
GPU is available


## GCN Model

### Load data

In [2]:
# Load data
# dataset = Citation(data, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])
imdb = tf.keras.datasets.imdb
(pp_train_data, pp_train_labels), (pp_test_data, pp_test_labels) = (
    imdb.load_data(num_words=10000))



In [3]:
# This block limits how much is loaded to keep debugging short
# don't run on the final experiments
validation_fraction = 0.2
limit = 25000

pp_validation_data = pp_train_data[0: int(limit * validation_fraction)]
pp_validation_labels = pp_train_labels[0: int(limit * validation_fraction)]
pp_train_data = pp_train_data[int(limit * validation_fraction): limit]
pp_train_labels = pp_train_labels[int(limit * validation_fraction): limit]
pp_test_data = pp_test_data[0:limit]
pp_test_labels = pp_test_labels[0:limit]

In [4]:
print('Training entries: {}, labels: {}'.format(
    len(pp_train_data), len(pp_train_labels)))
training_samples_count = len(pp_train_data)

Training entries: 20000, labels: 20000


In [5]:
print('Validation entries: {}, labels: {}'.format(
    len(pp_validation_data), len(pp_validation_labels)))

Validation entries: 5000, labels: 5000


In [6]:
print('Test entries: {}, labels: {}'.format(
    len(pp_test_data), len(pp_test_labels)))

Test entries: 25000, labels: 25000


In [7]:
print(pp_train_data[0])

[1, 13, 104, 14, 2856, 77, 6, 542, 20, 5, 49, 7, 94, 531, 26, 608, 21, 11, 226, 45, 6, 503, 20, 172, 44, 4, 114, 531, 26, 608, 21, 12, 47, 450, 1512, 5, 2, 15, 152, 179, 763, 56, 116, 9, 669, 608, 207, 110, 433, 7, 14, 99, 10, 10, 4, 454, 732, 4, 651, 19, 1880, 1953, 11, 4, 2092, 231, 646, 21, 484, 1744, 2980, 143, 49, 392, 2, 11, 550, 1270, 646, 40, 4, 9331, 69, 115, 110, 6439, 159, 12, 214, 1076, 128, 39, 50, 21, 24, 76, 10, 10, 48, 335, 2146, 851, 14, 79, 160, 31, 99, 5, 361, 14, 31, 17, 6, 2272, 56, 401, 1398, 364, 5, 12, 238, 157, 18, 25]


In [8]:
len(pp_train_data[0]), len(pp_train_data[1])

(124, 171)

In [9]:
def sampleFunction(inputFeature):
    """
    This is a description of the function
    
    Args:
        inputFeature - (np.ndarray) This is what the feature is
    Returns:
        result - (int) This is what is returned
    """

    result = 55
    return result

def buildReverseWordIndex(dataset):
    """
    Convert the index back to words with proper accounting for 
    the special characters reserved at the beginning of the dictionary

    Args: 
        dataset - (keras.dataset) The dataset to use
    Returns:
        buildReverseWordIndex - (dict) A dictionary mapping words to an integer index
    """
    wordIndex = dataset.get_word_index()

    # The first indices are reserved
    wordIndex = {k: (v + 3) for k, v in wordIndex.items()}
    wordIndex['<PAD>'] = 0
    wordIndex['<START>'] = 1
    wordIndex['<UNK>'] = 2  # unknown
    wordIndex['<UNUSED>'] = 3
    return dict((value, key) for (key, value) in wordIndex.items())

def decodeReview(text, reverseWordIndex):
    """
    Uses build_reverse_word_index to decode original data format into text
    
    Args:
        text - (np.ndarray) The text to decode
        reverseWordIndex - (dict) The reverse word index to use
    Returns:
        decodedReview - (string) The decoded review
    """
    return ' '.join([reverseWordIndex.get(i, '?') for i in text])


In [10]:
reverseWordIndex = buildReverseWordIndex(imdb)
decodeReview(pp_train_data[0], reverseWordIndex)

"<START> i think this could've been a decent movie and some of its parts are ok but in whole it's a b movie same about the plot parts are ok but it has several holes and <UNK> that doesn't quite add up acting is mostly ok i've seen worse of this too br br the beginning sets the level with cars driving in the desert making cool but totally unnecessary jumps through some small <UNK> in slow motion cool like the drivers had never seen sand before it gets slightly better from there but not much br br if you're gonna rent this get another one too and use this one as a warm up keep expectations low and it might work for you"

### Generate BERT Embedding

In [13]:
pretrained_embedding = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'

In [14]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

In [15]:
encoder_inputs = preprocessor(text_input)

encoder = hub.KerasLayer(
    'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2',
    trainable=True)

outputs = encoder(encoder_inputs)

pooled_output = outputs['pooled_output'] # [batch_size, 128].
# [batch_size, seq_length, 128].

sequence_output = outputs['sequence_output']

In [16]:
def int64Feature(value):
    """
    Returns int64 tf.train.Feature.

    Args:
        value - (np.ndarray) array of ints
    """
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value.tolist()))


def bytesFeature(value):
    """
    Returns bytes tf.train.Feature.

    Args:
        value - (string) string
    """
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))


def floatFeature(value):
    """
    Returns float tf.train.Feature.

    Args:
        value - (np.ndarray) array of floats

    """
    return tf.train.Feature(float_list=tf.train.FloatList(value=value.tolist()))


In [17]:
def createBertEmbeddingExample(wordVector, recordID, reverseWordIndex, encoder, preprocessor):
    """
    Create tf.Example containing the sample's embedding and its ID.
    
    Args:
        wordVector - (np.ndarray) the text to decode
        recordId - (int) ID of the sample
        reverseWordIndex - (dict) The reverse word index to use
        encoder - (string) encoder name
        preprocessor - (string) preprocessor name
    Returns:
        example - (tf.Example) tf.Example containing the sample's embedding and its ID
    """

    text = decodeReview(wordVector, reverseWordIndex)

    # Shape = [batch_size,].
    sentenceEmbedding = encoder(preprocessor(tf.reshape(text, shape=[-1, ])))['pooled_output']
    
    # Flatten the sentence embedding back to 1-D.
    sentenceEmbedding = tf.reshape(sentenceEmbedding, shape=[-1])
    
    features = {
        'id': bytesFeature(str(recordID)),
        'embedding': floatFeature(sentenceEmbedding.numpy())
    }
    return tf.train.Example(features=tf.train.Features(feature=features))


def createBertEmbedding(wordVectors, outputPath, startingRecordId, reverseWordIndex, encoder, preprocessor):
    """
    Create full set of BERT embeddings

    Args:
        wordVectors - (np.ndarray) all text to decode
        outputPath - (string) path to output file
        startingRecordId - (int) ID of the first sample
        reverseWordIndex - (dict) The reverse word index to use
        encoder - (string) encoder name
        preprocessor - (string) preprocessor name
    Returns:
        recordID - (int) ID of the last sample
    """
    recordID = int(startingRecordId)
    with tf.io.TFRecordWriter(outputPath) as writer:
        for word_vector in wordVectors:
            example = createBertEmbeddingExample(word_vector, recordID, reverseWordIndex, encoder, preprocessor)
            recordID = recordID + 1
            writer.write(example.SerializeToString())
    return recordID


In [ ]:
# !mkdir ./imdb

In [19]:
# Generate full BERT embeddings

bertEmbeddingsPath = './imdb/bertEmeddings.tfr'
createBertEmbedding(pp_train_data, bertEmbeddingsPath, 0, reverseWordIndex, encoder, preprocessor)

bertEmbeddingsPath_validation = './imdb/bertEmeddings_validation.tfr'
createBertEmbedding(pp_validation_data, bertEmbeddingsPath_validation, 0, reverseWordIndex, encoder, preprocessor)

bertEmbeddingsPath_test = './imdb/bertEmeddings_test.tfr'
createBertEmbedding(pp_test_data, bertEmbeddingsPath_test, 0, reverseWordIndex, encoder, preprocessor)
bertEmbeddingsPath

'./imdb/bertEmeddings.tfr'

In [21]:
!wc -l ./imdb/bertEmeddings.tfr

186718 ./imdb/bertEmeddings.tfr


In [20]:
def createBertEmbeddingExample_tf(wordVector, reverseWordIndex, encoder, preprocessor):
    """
    Create tf.Example containing the sample's embedding and its ID.
    
    Args:
        wordVector - (np.ndarray) the text to decode
        recordId - (int) ID of the sample
        reverseWordIndex - (dict) The reverse word index to use
        encoder - (string) encoder name
        preprocessor - (string) preprocessor name
    Returns:
        example - (tf.Example) tf.Example containing the sample's embedding and its ID
    """

    text = decodeReview(wordVector, reverseWordIndex)

    # Shape = [batch_size,].
    sentenceEmbedding = encoder(preprocessor(tf.reshape(text, shape=[-1, ])))['pooled_output']
    
    # Flatten the sentence embedding back to 1-D.
    sentenceEmbedding = tf.reshape(sentenceEmbedding, shape=[-1])
    
    # features = {
    #     'id': bytesFeature(str(recordID)),
    #     'embedding': floatFeature(sentenceEmbedding.numpy())
    # }
    # return tf.train.Example(features=tf.train.Features(feature=features))
    return sentenceEmbedding

def createBertEmbedding_tf(wordVectors, reverseWordIndex, encoder, preprocessor):
    """
    Create full set of BERT embeddings

    Args:
        wordVectors - (np.ndarray) all text to decode
        outputPath - (string) path to output file
        startingRecordId - (int) ID of the first sample
        reverseWordIndex - (dict) The reverse word index to use
        encoder - (string) encoder name
        preprocessor - (string) preprocessor name
    Returns:
        recordID - (int) ID of the last sample
    """
    sentenceEmbeddings = []
    for word_vector in wordVectors:
        example = createBertEmbeddingExample_tf(word_vector, reverseWordIndex, encoder, preprocessor)
        sentenceEmbeddings.append(example)
    sentenceEmbeddings_np = np.array(sentenceEmbeddings)
    return sentenceEmbeddings_np


In [21]:
bertEmbeddings_np = createBertEmbedding_tf(pp_train_data, reverseWordIndex, encoder, preprocessor)
bertEmbeddings_np

bertEmbeddings_va_np = createBertEmbedding_tf(pp_validation_data, reverseWordIndex, encoder, preprocessor)
bertEmbeddings_te_np = createBertEmbedding_tf(pp_test_data, reverseWordIndex, encoder, preprocessor)


In [22]:
np.shape(bertEmbeddings_np)

(20000, 128)

### Construct Graph Using BERT Embedding

In [23]:
similarity_threshold = 0.99

In [24]:

graph_builder_config = nsl.configs.GraphBuilderConfig(
    similarity_threshold=similarity_threshold, lsh_splits=32, lsh_rounds=15, random_seed=42)
nsl.tools.build_graph_from_config(['./imdb/bertEmeddings.tfr'],
                                  './imdb/graph_99.tsv',
                                  graph_builder_config)

In [25]:
graph_builder_config = nsl.configs.GraphBuilderConfig(
    similarity_threshold=similarity_threshold, lsh_splits=32, lsh_rounds=15, random_seed=42)
nsl.tools.build_graph_from_config(['./imdb/bertEmeddings_test.tfr'],
                                  './imdb/graph_99_test.tsv',
                                  graph_builder_config)

In [26]:
graph_builder_config = nsl.configs.GraphBuilderConfig(
    similarity_threshold=similarity_threshold, lsh_splits=32, lsh_rounds=15, random_seed=42)
nsl.tools.build_graph_from_config(['./imdb/bertEmeddings_validation.tfr'],
                                  './imdb/graph_99_validation.tsv',
                                  graph_builder_config)

In [29]:
!wc -l ./imdb/graph_99.tsv

24066 ./imdb/graph_99.tsv


### Convert NSL Graph to Spektral Graph



In [27]:
df = pd.read_csv('./imdb/graph_99.tsv', sep="\t") 
imdb_graph_a_adjacency_matrix = df.values
size = len(pp_train_data)
imdb_graph_a = np.zeros((size, size))
for row in imdb_graph_a_adjacency_matrix:
  imdb_graph_a[int(row[0]),int(row[1])] = row[2]


In [28]:
df = pd.read_csv('./imdb/graph_99_validation.tsv', sep="\t") 
imdb_graph_va_a_adjacency_matrix = df.values
size = len(pp_validation_data)
imdb_graph_va_a = np.zeros((size, size))
for row in imdb_graph_va_a_adjacency_matrix:
  imdb_graph_va_a[int(row[0]),int(row[1])] = row[2]


In [29]:
df = pd.read_csv('./imdb/graph_99_test.tsv', sep="\t") 
imdb_graph_te_a_adjacency_matrix = df.values
size = len(pp_test_data)
imdb_graph_te_a = np.zeros((size, size))
for row in imdb_graph_te_a_adjacency_matrix:
  imdb_graph_te_a[int(row[0]),int(row[1])] = row[2]


In [30]:
max_seq_length_slice = 256
# pp_train_data_sliced = np.zeros((len(pp_train_data), max_seq_length_slice))
# i = 0
# while i < len(pp_train_data):
#   if len(pp_train_data[i]) > max_seq_length_slice:
#     pp_train_data_sliced[i] = pp_train_data[i][:max_seq_length_slice]
#   else:
#     pp_train_data_sliced[i] = np.append(np.array(pp_train_data[i]), np.zeros(max_seq_length_slice - len(pp_train_data[i])))
#   i = i + 1




In [31]:
print(len(bertEmbeddings_np),len(bertEmbeddings_np[0]))
print(bertEmbeddings_np.shape)

20000 128
(20000, 128)


In [32]:
pp_train_labels_binary = np.zeros((len(pp_train_labels),2))
j = 0
for _ in pp_train_labels:
  if _ == 1:
    pp_train_labels_binary[j][1] = 1
  else:
    pp_train_labels_binary[j][0] = 1
  j = j + 1


In [33]:
len(pp_validation_labels)

5000

In [34]:
pp_validation_labels_binary = np.zeros((len(pp_validation_labels),2))
j = 0
for _ in pp_validation_labels:
  if _ == 1:
    pp_validation_labels_binary[j][1] = 1
  else:
    pp_validation_labels_binary[j][0] = 1
  j = j + 1


In [35]:
pp_test_labels_binary = np.zeros((len(pp_test_labels),2))
j = 0
for _ in pp_test_labels:
  if _ == 1:
    pp_test_labels_binary[j][1] = 1
  else:
    pp_test_labels_binary[j][0] = 1
  j = j + 1


In [36]:
imdb_graph=Graph(a=imdb_graph_a, x=bertEmbeddings_np, y=pp_train_labels_binary)
imdb_graph_va=Graph(a=imdb_graph_va_a, x=bertEmbeddings_va_np, y=pp_validation_labels_binary)
imdb_graph_te=Graph(a=imdb_graph_te_a, x=bertEmbeddings_te_np, y=pp_test_labels_binary)



In [37]:
class MyDataset(Dataset):
    """
    A dataset of five random graphs.
    """
    def __init__(self, graph, **kwargs):
        # self.nodes = nodes
        # self.feats = feats
        self.graph = graph
        super().__init__(**kwargs)

    def download(self):
        # data = ...  # Download from somewhere
        path = './imdb'
        # Create the directory
        
        # os.mkdir(path)
        # filename = os.path.join(path, 'imdb_graph')
        # np.savez(filename, x=imdb_graph.x, a=imdb_graph.a, y=imdb_graph.y)

    def read(self):
        # We must return a list of Graph objects
        output = []

        # for i in range(5):
        #     data = np.load(os.path.join(self.path, f'graph_{i}.npz'))
        #     output.append(
        #         Graph(x=data['x'], a=data['a'], y=data['y'])
        #     )
        output.append(self.graph)
        return output           

In [38]:
dataset = MyDataset(imdb_graph)
dataset[0]
dataset.apply(GCNFilter())

dataset_va = MyDataset(imdb_graph_va)
dataset_va.apply(GCNFilter())

dataset_te = MyDataset(imdb_graph_te)
dataset_te.apply(GCNFilter())

In [39]:
dataset[0].x.shape
dataset_va[0].x.shape
dataset_te[0].x.shape

(25000, 128)

### Train the model

In [40]:
loader_tr = SingleLoader(dataset)
loader_va = SingleLoader(dataset_va)

In [41]:
from tensorflow.keras.optimizers import SGD
learning_rate = 1
seed = 0
epochs = 1000
patience = 10

tf.random.set_seed(seed=seed)  # make weight initialization reproducible

In [42]:
from spektral.layers.convolutional import gcn_conv


class myGCN(tf.keras.Model):
    """
    This model, with its default hyperparameters, implements the architecture
    from the paper:
    > [Semi-Supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907)<br>
    > Thomas N. Kipf and Max Welling
    **Mode**: single, disjoint, mixed, batch.
    **Input**
    - Node features of shape `([batch], n_nodes, n_node_features)`
    - Weighted adjacency matrix of shape `([batch], n_nodes, n_nodes)`
    **Output**
    - Softmax predictions with shape `([batch], n_nodes, n_labels)`.
    **Arguments**
    - `n_labels`: number of channels in output;
    - `channels`: number of channels in first GCNConv layer;
    - `activation`: activation of the first GCNConv layer;
    - `output_activation`: activation of the second GCNConv layer;
    - `use_bias`: whether to add a learnable bias to the two GCNConv layers;
    - `dropout_rate`: `rate` used in `Dropout` layers;
    - `l2_reg`: l2 regularization strength;
    - `**kwargs`: passed to `Model.__init__`.
    """

    def __init__(
        self,
        n_labels,
        channels=256,
        activation="relu",
        output_activation="softmax",
        use_bias=False,
        dropout_rate=0.3,
        l2_reg=2.5e-4,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.n_labels = n_labels
        self.channels = channels
        self.activation = activation
        self.output_activation = output_activation
        self.use_bias = use_bias
        self.dropout_rate = dropout_rate
        self.l2_reg = l2_reg
        reg = tf.keras.regularizers.l2(l2_reg)
        self._d0 = tf.keras.layers.Dropout(dropout_rate)
        self._gcn0 = gcn_conv.GCNConv(
            channels, activation=activation, kernel_regularizer=reg, use_bias=use_bias
        )
        self._d1 = tf.keras.layers.Dropout(dropout_rate)
        self._gcn1 = gcn_conv.GCNConv(
            n_labels, activation=output_activation, use_bias=use_bias
        )

    def get_config(self):
        return dict(
            n_labels=self.n_labels,
            channels=self.channels,
            activation=self.activation,
            output_activation=self.output_activation,
            use_bias=self.use_bias,
            dropout_rate=self.dropout_rate,
            l2_reg=self.l2_reg,
        )

    def call(self, inputs):
        if len(inputs) == 2:
            x, a = inputs
        else:
            x, a, _ = inputs  # So that the model can be used with DisjointLoader

        x = self._d0(x)
        x = self._gcn0([x, a])
        x = self._d1(x)
        return self._gcn1([x, a])

In [43]:
model = myGCN(n_labels=imdb_graph.n_labels)

In [44]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(),
    weighted_metrics=["acc"],
)

In [45]:
# Train model
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data = loader_va.load(),
    validation_steps = loader_va.steps_per_epoch,
    epochs=epochs,
    # callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
    verbose=1
)

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.8505 - acc: 0.4961 - val_loss: 2.8100 - val_acc: 0.4908
Epoch 2/1000
1/1 [==============================] - 0s 156ms/step - loss: 2.8160 - acc: 0.5023 - val_loss: 0.7531 - val_acc: 0.4908
Epoch 3/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.7863 - acc: 0.5114 - val_loss: 0.7327 - val_acc: 0.5254
Epoch 4/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.7395 - acc: 0.5008 - val_loss: 0.7331 - val_acc: 0.5508
Epoch 5/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.7375 - acc: 0.5072 - val_loss: 0.7329 - val_acc: 0.5582
Epoch 6/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.7362 - acc: 0.5120 - val_loss: 0.7325 - val_acc: 0.5674
Epoch 7/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.7349 - acc: 0.5228 - val_loss: 0.7320 - val_acc: 0.5702
Epoch 8/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.7344 - ac

Epoch 125/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.7014 - acc: 0.6132 - val_loss: 0.6989 - val_acc: 0.6194
Epoch 126/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.6965 - acc: 0.6166 - val_loss: 0.6918 - val_acc: 0.6250
Epoch 127/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.6942 - acc: 0.6206 - val_loss: 0.6888 - val_acc: 0.6230
Epoch 128/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6940 - acc: 0.6166 - val_loss: 0.6863 - val_acc: 0.6252
Epoch 129/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6936 - acc: 0.6166 - val_loss: 0.6849 - val_acc: 0.6262
Epoch 130/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.6921 - acc: 0.6188 - val_loss: 0.6848 - val_acc: 0.6248
Epoch 131/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6930 - acc: 0.6177 - val_loss: 0.6836 - val_acc: 0.6266
Epoch 132/1000
1/1 [==============================] - 0s 18ms/step - 

Epoch 248/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6841 - acc: 0.6236 - val_loss: 0.6771 - val_acc: 0.6260
Epoch 249/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6853 - acc: 0.6224 - val_loss: 0.6699 - val_acc: 0.6422
Epoch 250/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6867 - acc: 0.6230 - val_loss: 0.6879 - val_acc: 0.6166
Epoch 251/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.6918 - acc: 0.6062 - val_loss: 0.6750 - val_acc: 0.6322
Epoch 252/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.6961 - acc: 0.6045 - val_loss: 0.7061 - val_acc: 0.5854
Epoch 253/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.7016 - acc: 0.5918 - val_loss: 0.6837 - val_acc: 0.6310
Epoch 254/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6975 - acc: 0.5950 - val_loss: 0.6886 - val_acc: 0.6212
Epoch 255/1000
1/1 [==============================] - 0s 18ms/step - 

Epoch 371/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6822 - acc: 0.6221 - val_loss: 0.6675 - val_acc: 0.6448
Epoch 372/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6830 - acc: 0.6224 - val_loss: 0.6770 - val_acc: 0.6230
Epoch 373/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6806 - acc: 0.6202 - val_loss: 0.6663 - val_acc: 0.6456
Epoch 374/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6816 - acc: 0.6227 - val_loss: 0.6792 - val_acc: 0.6192
Epoch 375/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6815 - acc: 0.6204 - val_loss: 0.6668 - val_acc: 0.6424
Epoch 376/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6828 - acc: 0.6180 - val_loss: 0.6798 - val_acc: 0.6188
Epoch 377/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.6818 - acc: 0.6184 - val_loss: 0.6674 - val_acc: 0.6422
Epoch 378/1000
1/1 [==============================] - 0s 21ms/step - 

Epoch 494/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6795 - acc: 0.6202 - val_loss: 0.6750 - val_acc: 0.6238
Epoch 495/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6764 - acc: 0.6242 - val_loss: 0.6632 - val_acc: 0.6442
Epoch 496/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.6764 - acc: 0.6281 - val_loss: 0.6720 - val_acc: 0.6266
Epoch 497/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.6773 - acc: 0.6222 - val_loss: 0.6619 - val_acc: 0.6450
Epoch 498/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.6747 - acc: 0.6272 - val_loss: 0.6721 - val_acc: 0.6260
Epoch 499/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6783 - acc: 0.6219 - val_loss: 0.6618 - val_acc: 0.6444
Epoch 500/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.6762 - acc: 0.6241 - val_loss: 0.6746 - val_acc: 0.6220
Epoch 501/1000
1/1 [==============================] - 0s 24ms/step - 

Epoch 617/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6751 - acc: 0.6198 - val_loss: 0.6593 - val_acc: 0.6420
Epoch 618/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6727 - acc: 0.6237 - val_loss: 0.6713 - val_acc: 0.6268
Epoch 619/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.6732 - acc: 0.6230 - val_loss: 0.6591 - val_acc: 0.6418
Epoch 620/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6751 - acc: 0.6223 - val_loss: 0.6718 - val_acc: 0.6250
Epoch 621/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6755 - acc: 0.6205 - val_loss: 0.6600 - val_acc: 0.6414
Epoch 622/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.6754 - acc: 0.6212 - val_loss: 0.6701 - val_acc: 0.6286
Epoch 623/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6738 - acc: 0.6211 - val_loss: 0.6594 - val_acc: 0.6452
Epoch 624/1000
1/1 [==============================] - 0s 26ms/step - 

Epoch 740/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6701 - acc: 0.6259 - val_loss: 0.6651 - val_acc: 0.6340
Epoch 741/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6688 - acc: 0.6258 - val_loss: 0.6547 - val_acc: 0.6450
Epoch 742/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6700 - acc: 0.6271 - val_loss: 0.6660 - val_acc: 0.6312
Epoch 743/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6706 - acc: 0.6201 - val_loss: 0.6557 - val_acc: 0.6424
Epoch 744/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.6727 - acc: 0.6201 - val_loss: 0.6679 - val_acc: 0.6274
Epoch 745/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6713 - acc: 0.6216 - val_loss: 0.6560 - val_acc: 0.6434
Epoch 746/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6709 - acc: 0.6234 - val_loss: 0.6665 - val_acc: 0.6328
Epoch 747/1000
1/1 [==============================] - 0s 22ms/step - 

Epoch 863/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6662 - acc: 0.6254 - val_loss: 0.6516 - val_acc: 0.6474
Epoch 864/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6684 - acc: 0.6245 - val_loss: 0.6642 - val_acc: 0.6304
Epoch 865/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6677 - acc: 0.6249 - val_loss: 0.6526 - val_acc: 0.6456
Epoch 866/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6666 - acc: 0.6263 - val_loss: 0.6642 - val_acc: 0.6312
Epoch 867/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6686 - acc: 0.6226 - val_loss: 0.6531 - val_acc: 0.6442
Epoch 868/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6691 - acc: 0.6264 - val_loss: 0.6632 - val_acc: 0.6336
Epoch 869/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.6673 - acc: 0.6244 - val_loss: 0.6523 - val_acc: 0.6448
Epoch 870/1000
1/1 [==============================] - 0s 24ms/step - 

Epoch 986/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6643 - acc: 0.6280 - val_loss: 0.6606 - val_acc: 0.6342
Epoch 987/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.6656 - acc: 0.6263 - val_loss: 0.6500 - val_acc: 0.6452
Epoch 988/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.6678 - acc: 0.6230 - val_loss: 0.6626 - val_acc: 0.6316
Epoch 989/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.6658 - acc: 0.6274 - val_loss: 0.6505 - val_acc: 0.6464
Epoch 990/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.6655 - acc: 0.6255 - val_loss: 0.6603 - val_acc: 0.6332
Epoch 991/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.6650 - acc: 0.6299 - val_loss: 0.6494 - val_acc: 0.6464
Epoch 992/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.6635 - acc: 0.6270 - val_loss: 0.6590 - val_acc: 0.6336
Epoch 993/1000
1/1 [==============================] - 0s 22ms/step - 

In [46]:
model.summary()

Model: "my_gcn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           multiple                  0         
                                                                 
 gcn_conv (GCNConv)          multiple                  32768     
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 gcn_conv_1 (GCNConv)        multiple                  512       
                                                                 
Total params: 33,280
Trainable params: 33,280
Non-trainable params: 0
_________________________________________________________________


### Evaluate Model

In [133]:
# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 439ms/step - loss: 0.6348 - acc: 0.6530
Done.
Test loss: 0.6348326206207275
Test accuracy: 0.6529600024223328
